# Python Exercises

#### Written for the QuantEcon Africa Workshop (July 2023)
#### Author: [John Stachurski](http://johnstachurski.net/)

Before you attempt these exercises, we recommend that you read

1. the [lecture on NumPy](https://python-programming.quantecon.org/numpy.html),
2. the [lecture on Matplotlib](https://python-programming.quantecon.org/matplotlib.html) and
3. the [lecture on SciPy](https://python-programming.quantecon.org/scipy.html).


## Exercises

### Exercise 1

In this exercise, 

1. Draw 1000 independent draws from the standard normal distribution using [`scipy.stats`](https://docs.scipy.org/doc/scipy/reference/stats.html).

2. Calculate the sample mean and variance of the draws and compare them to the theoretical values.

3. Visualize the empirical distribution of the draws using a histogram. Mark the sample mean and $\pm$ one standard deviation using vertical lines.

4. Calculate the probability that a draw from the distribution is less than 0 and compare it to the proportion of the sample that is less than 0.

5. How do the results in 3 change with sample size? Try to draw 10, 100, 500, 1000, 10000 samples 100 times each and plot the results.

Use the following imports to get started.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm

### Solution

**Exercise 1.1**

In [ ]:
# Define the parameters of the standard normal distribution
mu = 0
sigma = 1

# Generate a sample of 10000 draws from the normal distribution
samples = norm.rvs(mu, sigma, size=10_000)

**Exercise 1.2**

In [ ]:
# Calculate the mean and standard deviation of the sample
mean = np.mean(samples)
std = np.std(samples)

print("Sample mean: {:.4f}".format(mean))
print("Theoretical mean: {:.4f}".format(mu))
print("Sample standard deviation: {:.4f}".format(std))
print("Theoretical standard deviation: {:.4f}".format(sigma))


**Exercise 1.3**

In [ ]:
# Plot the histogram of the sample
plt.hist(samples, bins=100)
plt.axvline(mean, color='red')
plt.axvline(mean + std, color='grey')
plt.axvline(mean - std, color='grey')
plt.show()

**Exercise 1.4**

In [ ]:
# Calculate the probability that a draw from the distribution is less than 0
probability = norm.cdf(0, mu, sigma)

print("The probability that a draw from the distribution is less than 0 is:", probability)

# Calculate the proportion of samples is less than 0
proportion = np.mean(samples < 0)

print("The proportion of samples is less than 0 is:", proportion)

**Exercise 1.5**

In [ ]:
# Create a function that draws n samples from 
# the standard normal distribution and returns 
# mean and starndard deviation
def draw_samples(n):
    samples = norm.rvs(mu, sigma, size=n)
    mean = np.mean(samples)
    std = np.std(samples)
    return mean, std

sample_sizes = [10, 100, 500, 1000, 10_000]
sizes_arr = np.repeat(sample_sizes, 100)

# Draw 100 samples for each sample size using vectorized numpy code
means, stds = np.vectorize(draw_samples)(sizes_arr)
means, stds = means.reshape(5, 100), stds.reshape(5, 100)

for i in range(5):
    # Plot scatter plot of means and standard deviations
    plt.scatter(means[i], stds[i], alpha=0.5, 
                label="n = {}".format(sample_sizes[i]))
    plt.axhline(sigma, color='red')
    plt.axvline(mu, color='red')
    plt.xlabel("Sample mean")
    plt.ylabel("Sample standard deviation")
    plt.legend()
    # Limit the axis to zoom in on the plot
    plt.xlim(-0.5, 0.5)
    plt.ylim(0.5, 1.5)


### Exercise 2

Simulate and plot the correlated time series

$$
    x_{t+1} = \alpha \, x_t + \epsilon_{t+1}
    \quad \text{where} \quad
    x_0 = 0 
    \quad \text{and} \quad t = 0,\ldots,T
$$

Here $\{\epsilon_t\}$ is iid and standard normal.

In your solution, restrict your import statements to

In [ ]:
from random import normalvariate
import matplotlib.pyplot as plt

Set $T=200$ and $\alpha = 0.9$

### Solution

In [ ]:
import random
# Set random seed to replicate solution
# (This is optional and used so that each time you run the cell, you get same results)
random.seed(2023)

alpha = 0.9
ts_length = 200
x = 0

x_values = []
for i in range(ts_length):
    x_values.append(x)
    x = alpha * x + normalvariate(0, 1)
fig, ax = plt.subplots()
ax.plot(x_values, '-')
plt.show()

### Exercise 3

Generate 100000 data points from the [exponential distribution](https://en.wikipedia.org/wiki/Exponential_distribution) with density

$$
f(x; \alpha) = \alpha \exp(-\alpha x)
\qquad
(x > 0, \alpha > 0)
$$

taking $\alpha = 0.5$. Then

1. Plot a histogram of your sample and compare it to the density of the exponential distribution.
2. After looking up the maximum likelihood estimator of $\alpha$, compute the estimate given your data and check that it is in fact close to $\alpha$.

### Solution

After checking [the docs for the exponential distribution](http://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.expon.html) we proceed as follows

In [ ]:
from scipy.stats import expon
import numpy as np

alpha = 0.5
n = int(1e5)
# Scale controls the exponential parameter
ep = expon(scale=1.0/alpha)
# Generate n randome variables
x = ep.rvs(size=n)

Here's a histogram and density.

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))
xmin, xmax = 0.001, 10.0
ax.set_xlim(xmin, xmax)
ax.hist(x, density=True, bins=60, alpha=0.3)
grid = np.linspace(xmin, xmax, 200)
ax.plot(grid, ep.pdf(grid), 'k-', lw=2, label='true density')
ax.legend()
plt.show()

It's [well-known](http://en.wikipedia.org/wiki/Exponential_distribution) that the MLE of $\alpha$ is $1/\bar x$ where $\bar x$ is the mean of the sample.  Let's check that it is indeed close to $\alpha$.

In [ ]:
alpha_mle = 1.0 / x.mean()
print(f"max likelihood estimate of alpha is {alpha_mle}.")

### Exercise 4

Using the same data set, implement maximum likelihood again, but this time pretending that you don't know the analytical expression for the maximum likelihood estimator.  Instead, set up the log likelihood function and maximize it numerically using a routine from `scipy.optimize`.

(Hint: Have a look at the optimization examples from the scientific Python quickstart notebook.)

### Solution

First let's set up the log likelihood function.

In [ ]:
s = x.sum()
def neg_loglike(a):
    return - n * np.log(a) + a * s

This function is minus the log likelihood function for the exponential distribution.

Minimize over a reasonable parameter space

In [ ]:
from scipy.optimize import minimize_scalar
res = minimize_scalar(neg_loglike, bounds=(0.01, 10.0), method='bounded')
res.x

This is close to the analytical value of the max likelihood estimator we got in exercise 2.

### Exercise 5

Recall that a discrete Lyapunov equation is a matrix equation of the form


\begin{equation}
    X = A X A' + M
\end{equation}


Here all matrices are $n \times n$ and $X$ is the unknown.  $A'$ is the transpose of $A$.  The equation has a unique solution if the spectral radius of $A$ is less than 1.

There is a solver for Lyapunov equations in SciPy.  Let's try it out with these matrices:

In [ ]:
import numpy as np
A = np.array([[0, 1],[-1/2, -1]])
M = np.array([[0, 0], [0, 9]])

In [ ]:
A

In [ ]:
M

Here's the solver and the solution.

In [ ]:
from scipy.linalg import solve_discrete_lyapunov
solve_discrete_lyapunov(A, M)

In fact it's possible to obtain this solution by iteration, starting with a guess $X_0$, such as $X_0 = M$, and then iterating on

$$
    X_{n+1} = A X_n A' + M
$$

Try to obtain the same solution using an iterative scheme.  (That is, start with $X_0$, then compute $X_1$, then $X_2$, etc.  You can stop when $X_{n+1}$ and $X_n$ are close, or by using some other simpler method.  But check that you get a result close to the solution above.)

### Solution

Here's an iterative algorithm that computes the solution.

In [ ]:
P = M
tol = 1e-6
max_iter = 500

for i in range(max_iter):
    P_new = A @ P @ A.T + M
    error = np.linalg.norm(P - P_new, ord=2)
    if error < tol:
        break
    P = P_new

P

This is close to what we had before:

In [ ]:
solve_discrete_lyapunov(A, M)